# GIS module

GIS operations are integral to hydrology processes. This page demonstrates how to use `xhydro` to perform GIS manipulations such as delineating watershed boundaries and extracting physiographic, climatological and geographical variables at the watershed scale.

In [ ]:
import leafmap
import pandas as pd
import xarray as xr
import xclim
import xdatasets as xd
from IPython.core.display import HTML, display

import xhydro.gis as xhgis
from xhydro.indicators import get_yearly_op

## Watershed delineation

Currently, watershed delineation uses HydroBASINS (hybas_na_lev01-12_v1c) and can work in any location in North America. The process involves assessing all upstream sub-basins from a specified outlet and consolidating them into a unified watershed. The [leafmap](https://leafmap.org/) library is employed for generating interactive maps. This map serves the purpose of selecting outlets or visualizing the resulting watershed boundaries. Although utilizing the map is not essential for conducting the calculations, it proves useful for visualization purposes.

In [ ]:
m = leafmap.Map(center=(48.63, -74.71), zoom=5, basemap="USGS Hydrography")
m

### a) From a list of coordinates
In this scenario, we select two pour points, with each one representing the outlet for the watersheds of Lac Saint-Jean and the Ottawa River, respectively.

In [ ]:
lng_lat = [
    (-69.81971, 48.14720),  # Lac Saint-Jean watershed
    (-74.393438, 45.572442),  # Ottawa river watershed
]

### b) From markers on a map

Instead of using a list, a more interactive approach is to directly select outlets from the existing map ``m``. The following image illustrates the process of selecting pour points by dragging markers to the desired locations on the map.

![test](../../docs/_static/_images/example_draw_marker.png)

The next cell is only useful for the documentation as it simulates a user selecting an outlet from the map `m`. You should instead remove this code and interact with the map in object `m` as shown above by positionning markers at sites of interest

In [ ]:
m.draw_features = [
    {
        "type": "Feature",
        "properties": {},
        "geometry": {"type": "Point", "coordinates": [-73.118597, 46.042467]},
    }
]  # Richelieu watershed

After selecting points using either approach a) or b), or a combination of both, we can initiate the watershed delineation calculation.

In [ ]:
gdf = xhgis.watershed_delineation(coordinates=lng_lat, map=m)
gdf

The outcomes are stored in a GeoPandas `gpd.GeoDataFrame` (`gdf`) object, allowing us to save our polygons in various common formats such as an ESRI Shapefile or GeoJSON. If a map ``m`` is present, the polygons will automatically be added to it. If you want to visualize the map, simply type ``m`` in the code cell to render it. If displaying the map directly is not compatible with your notebook interpreter, you can utilize the following code to extract the HTML from the map and plot it:

In [ ]:
m.zoom_to_gdf(gdf)

### c) From [xdatasets](https://github.com/hydrologie/xdatasets)

Automatically delineating watershed boundaries is a valuable tool in the toolbox, but users are encouraged to utilize official watershed boundaries if they already exist, instead of creating new ones. This functionality fetches a list of basins from [xdatasets](https://github.com/hydrologie/xdatasets) supported datasets, and upon request, [xdatasets](https://github.com/hydrologie/xdatasets) provides a `gpd.GeoDataFrame` containing the precalculated boundaries for these basins. Currently, the following watershed sources are available as of today.:

| Source                                                                                                                                                             | Dataset name   |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------|
| [DEH](https://www.cehq.gouv.qc.ca/atlas-hydroclimatique/stations-hydrometriques/index.htm)                                                                         | deh_polygons   |
| [HYDAT](https://www.canada.ca/en/environment-climate-change/services/water-overview/quantity/monitoring/survey/data-products-services/national-archive-hydat.html) | hydat_polygons |
| [HQ](https://www.hydroquebec.com/r)                                                                                                                                | hq_polygons    |


In [ ]:
gdf = xd.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["031*", "0421*"],
                "regulated": ["Natural"],
            }
        }
    }
).data.reset_index()

gdf

## Extract watershed properties

After obtaining our watershed boundaries, we can extract valuable properties such as geographical information, land use classification and climatological data from the delineated watersheds.

### a) Geographical watershed properties
Initially, we extract geographical properties of the watershed, including the perimeter, total area, Gravelius coefficient and basin centroid. It's important to note that this function returns all the columns present in the provided `gpd.GeoDataFrame` argument.

In [ ]:
xhgis.watershed_properties(gdf)

For added convenience, we can also retrieve the same results in the form of an `xarray.Dataset`:

In [ ]:
xhgis.watershed_properties(
    gdf[["Station", "geometry"]], unique_id="Station", output_format="xarray"
)

### b) Land-use classification
Land use classification is powered by the Planetary Computer's STAC catalog. It uses the `10m Annual Land Use Land Cover` dataset by default ("io-lulc-9-class"), but other collections can be specified by using the collection argument. 

In [ ]:
df = xhgis.land_use_classification(gdf, unique_id="Station")
df

In [ ]:
ax = xhgis.land_use_plot(gdf, unique_id="Station", idx=2)

### c) Climate indicators
The step of extracting climatic indicators is the most complex. Indeed, to accomplish this, access to a weather dataset for the various watersheds within our `gdf` object is required. Fortunately, `xdatasets` precisely facilitates such operations. Indeed, `xdatasets` allows extracting from a gridded dataset all the pixels contained within a watershed while respecting the weighting of the watershed intersecting each pixel.Subsequently, the function `get_yearly_op`, built upon the `xclim` library, offers impressive flexibility in defining indicators tailored to the user's needs.

To initiate the process, we employ ERA5-Land reanalysis data spanning the period from 1981 to 2010 as our climatological dataset.

In [ ]:
datasets = {
    "era5_land_reanalysis": {"variables": ["t2m", "tp", "sd"]},
}
space = {
    "clip": "polygon",  # bbox, point or polygon
    "averaging": True,
    "geometry": gdf,  # 3 polygons
    "unique_id": "Station",
}
time = {
    "start": "1981-01-01",
    "end": "2010-12-31",
    "timezone": "America/Montreal",
}

ds = xd.Query(datasets=datasets, space=space, time=time).data.squeeze()

Because the next few steps use [xclim](https://xclim.readthedocs.io/en/stable/index.html) under the hood, the dataset is required to be [CF-compliant](http://cfconventions.org/cf-conventions/cf-conventions.html). At a minimum, the `xarray.DataArray` used must follow these principles:

- The dataset needs a time dimension, usually at a daily frequency with no missing timesteps (NaNs are supported). If your data differs from that, you'll need to be extra careful on the results provided.
- If there is a spatial dimension, such as "``Station``" in the example below, it needs an attribute ``cf_role`` with ``timeseries_id`` as its value.
- The variable will at the very least need a ``units`` attribute, although other attributes such as ``long_name`` and ``cell_methods`` are also expected by `xclim` and warnings will be generated if they are missing.
- While this is not necessary for get_yearly_op, variable names should be one of those supported here for maximum compatibility.

The following code adds the missing attributes :

In [ ]:
ds = ds.rename({"t2m": "tas", "tp": "pr", "sd": "snd"})
ds["tas"] = xclim.core.units.convert_units_to(
    ds["tas"], "degC"
)  # Convert from degK to degC
ds["tas"].attrs.update({"cell_methods": "time: mean"})

ds["pr"].attrs.update({"units": "m d-1", "cell_methods": "time: mean within days"})
ds["pr"] = xclim.core.units.convert_units_to(
    ds["pr"], "mm d-1"
)  # Convert from m/d to mm/d

ds["snd"].attrs.update({"units": "m", "cell_methods": "time: mean within days"})
ds["snd"] = xclim.core.units.convert_units_to(ds["snd"], "mm")  # Convert from m to mm
ds

In the second step, we can define seasons using indexers that are compatible with ``xclim.core.calendar.select_time``. There are currently four accepted types of indexers:

- `month`, followed by a sequence of month numbers.

- `season`, followed by one or more of `‘DJF’`, `‘MAM’`, `‘JJA’`, and `‘SON’`.

- `doy_bounds`, followed by a sequence representing the inclusive bounds of the period to be considered (`'start'`, `'end'`).

- `date_bounds`, which is the same as above, but using a month-day (`'%m-%d'`) format.

Following this, we specify the operations we intend to calculate for each variable. The supported operations include `"max"`, `"min"`, `"mean"`, and `"sum"`.

In [ ]:
timeargs = {
    "01": {"month": [1]},
    "02": {"month": [2]},
    "03": {"month": [3]},
    "04": {"month": [4]},
    "05": {"month": [5]},
    "06": {"month": [6]},
    "07": {"month": [7]},
    "08": {"month": [8]},
    "09": {"month": [9]},
    "10": {"month": [10]},
    "11": {"month": [11]},
    "12": {"month": [12]},
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer_fall": {"date_bounds": ["06-20", "11-19"]},
    "year": {"date_bounds": ["01-01", "12-31"]},
}

operations = {
    "tas": ["max", "mean", "min"],
    "pr": ["sum"],
    "snd": ["mean"],
}

The combination of `timeargs` and `operations` through the Cartesian product yields a rapid generation of an extensive array of climate indicators.

In [ ]:
ds_climatology = xr.merge(
    [
        get_yearly_op(ds, input_var=variable, op=op, timeargs=timeargs)
        for (variable, ops) in operations.items()
        for op in ops
    ]
)
ds_climatology

The same data can also be visualized as a `pd.DataFrame` as well : 

In [ ]:
pd.set_option("display.max_rows", 100)
ds_climatology.mean("time").to_dataframe().T